<a href="https://colab.research.google.com/github/buganart/melgan-neurips/blob/master/Resample_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [3]:
#@markdown Check how many CPUs we have, you can choose a high memory instance to get 4.
import os
print(f"We have {os.cpu_count()} CPU cores.")

We have 4 CPU cores.


In [4]:
#@markdown Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

Mounted at /content/drive


In [5]:
#@markdown Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
audio_db_dir = "/content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material" #@param {type:"string"}
resample_dir = "/content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material Resampled" #@param {type:"string"}
sample_rate = 22050 #@param {type:"string"}
sample_rate = int(sample_rate)

audio_db_dir = Path(audio_db_dir)
resample_dir = Path(resample_dir)

resample_dir.mkdir(parents=True, exist_ok=True)

if not audio_db_dir.exists():
    raise RuntimeError("audio_db_dir {audio_db_dir} does not exists.")

In [ ]:
#@title
import subprocess
from pathlib import Path

from joblib import Parallel, delayed


def convert(input, output, sample_rate):
    command = ["ffmpeg", "-i", str(input), "-y", "-ar", str(sample_rate), str(output)]
    try:
        return subprocess.check_output(command, stderr=subprocess.STDOUT,)
    except subprocess.CalledProcessError as exc:
        print(f"Return code: {exc.returncode}\n", exc.output)
        raise


def main(*, in_dir, out_dir, sample_rate):
    in_dir = Path(in_dir)
    out_dir = Path(out_dir)
    in_paths = list(Path(in_dir).rglob("*.*"))
    out_paths = [out_dir / in_path.relative_to(in_dir) for in_path in in_paths]

    for sub_dir in set(out_path.parent for out_path in out_paths):
        sub_dir.mkdir(exist_ok=True, parents=True)

    Parallel(n_jobs=-1, verbose=2)(
        delayed(convert)(in_path, out_path, sample_rate)
        for in_path, out_path in zip(in_paths, out_paths)
    )

main(in_dir=audio_db_dir, out_dir=resample_dir, sample_rate=sample_rate)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   44.7s
